# Chains

- LLM
- Router
- Sequential
- Transformation

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## LLM
最基礎的Chain

In [4]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "請問{country}的首都在哪裡?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("日本")

{'country': '日本', 'text': '\n\n日本的首都是東京。'}

有一些額外的用法，`apply`允許輸入一個list

In [5]:
input_list = [
    {"country": "日本"},
    {"country": "韓國"},
    {"country": "北京"}  # 故意打錯試試看
]

llm_chain.apply(input_list)


[{'text': '\n\n日本的首都是東京。'},
 {'text': '\n\n韓國的首都是首爾。'},
 {'text': '\n\n北京的首都位於中國首都北京市。'}]

### 解析輸出

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """列出亞洲地區的國家，並使用comma分隔"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [13]:
llm_chain.predict_and_parse()

['中國',
 '日本',
 '韓國',
 '印度',
 '伊朗',
 '印尼',
 '泰國',
 '越南',
 '柬埔寨',
 '緬甸',
 '馬來西亞',
 '斯里蘭卡',
 '巴基斯坦',
 '菲律賓',
 '新加坡',
 '東帝汶',
 '尼泊爾',
 '哈薩克斯坦',
 '塔吉克斯坦',
 '不丹',
 '阿富汗',
 '亞美尼亞',
 '土耳其',
 '以色列',
 '伊拉克',
 '約旦',
 '黎巴嫩',
 '以色列',
 '伊拉克',
 '約旦',
 '黎巴']

## Router
RouterChain 可以根據輸入，來選擇列表中最適合作為下一個階段的chain

In [19]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [20]:
drug_template = """
You have a deep understanding of pharmaceutical knowledge and excel at answering questions.
Based on the inquiry, please provide information on the ingredients, uses, side effects, and precautions for this medication.
Here are the questions:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [21]:
prompt_infos = [
    {
        "name": "pharmaceutical",
        "description": "Good for answering questions about pharmaceutical",
        "prompt_template": drug_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]


In [22]:
llm = OpenAI()

In [24]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [25]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [26]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [27]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [28]:
print(chain.run("3的4次方是多少?請用繁體中文回答"))



> Entering new MultiPromptChain chain...


/Users/Viwave/ai-dev/langchain-practice/venv/lib/python3.10/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': '3的4次方是多少？請用繁體中文回答'}
> Finished chain.
。

3的4次方等於 81。


In [30]:
print(chain.run("阿司匹林是什麼?請用繁體中文回答"))



> Entering new MultiPromptChain chain...
pharmaceutical: {'input': '阿司匹林是什麼？請用繁體中文回答'}
> Finished chain.


阿司匹林是一種抗凝血的藥物，主要用於預防心臟病、腦中風和深静脈血栓的發生，也可用於治療不同程度的症狀。它的主要成分是水合氯醛，可以阻止血小板在血管中凝聚，防止血栓的形成。它可以與其他抗凝血藥物一起使用，以增強其抗凝血作用。

阿司匹林的副作用
